<a href="https://colab.research.google.com/github/Boonyaratt/Two-Tower-Recommendation/blob/master/LGBMRanker_with.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.Import Library & Datasets

In [1]:
# import os
# os.environ['TF_USE_LEGACY_KERAS'] = '1'
# os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
# os.environ['TF_DISABLE_MKL'] = '1'

In [2]:
# !pip -q install --upgrade pip setuptools wheel

# # 1) แกนหลักที่เข้าชุดกันบน Py3.12
# !pip -q install "numpy==1.26.4" "ml-dtypes==0.3.2"

# # 3) Install TensorFlow 2.16.1
# !pip -q install "tensorflow==2.16.1"

# # 4) Install TFRS 0.7.3
# !pip -q install "tensorflow-recommenders==0.7.3"

# !pip install holidays python-dateutil

In [3]:
# Import required libraries
import pandas as pd
import numpy as np

# import tensorflow as tf
# import tensorflow_recommenders as tfrs
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
from datetime import datetime, timedelta
from dateutil import easter
from typing import Dict, Text
import os, kagglehub
import lightgbm as lgb
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)

print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)


Pandas version: 2.2.2
NumPy version: 2.0.2


In [4]:
!mkdir Dataset

mkdir: cannot create directory ‘Dataset’: File exists


# Datasets

## 2.Display Dataset Information

In [5]:
import os, pathlib
os.environ["KAGGLEHUB_CACHE"] = "/content/Dataset"  # exact case
pathlib.Path("/content/Dataset").mkdir(parents=True, exist_ok=True)

import kagglehub
kaggle_path = kagglehub.dataset_download("frtgnn/dunnhumby-the-complete-journey")
print(kaggle_path)

Using Colab cache for faster access to the 'dunnhumby-the-complete-journey' dataset.
/kaggle/input/dunnhumby-the-complete-journey


In [6]:
# Read all CSV files

path = "/kaggle/input/dunnhumby-the-complete-journey/"

campaign_desc = pd.read_csv(path + "campaign_desc.csv")
campaign_table = pd.read_csv(path + "campaign_table.csv")
coupon_redempt = pd.read_csv(path + "coupon_redempt.csv") ## 1 counpon_unc can have multiple product_id: 556 nunique from 2318
coupon = pd.read_csv(path + "coupon.csv")
product = pd.read_csv(path + "product.csv")
transaction_data = pd.read_csv(path + "transaction_data.csv")
# Check the dataframes
print("DataFrames loaded successfully!")
print(f"campaign_desc shape: {campaign_desc.shape}")
print(f"campaign_table shape: {campaign_table.shape}")
print(f"coupon_redempt shape: {coupon_redempt.shape}")
print(f"coupon shape: {coupon.shape}")
print(f"product shape: {product.shape}")
print(f"transaction_data shape: {transaction_data.shape}")

# Show headers for each dataset
print("Headers for each dataset:")
print("=" * 50)

datasets = {
    'campaign_desc': campaign_desc,
    'campaign_table': campaign_table,
    'coupon_redempt': coupon_redempt,
    'coupon': coupon,
    'product': product,
    'transaction_data': transaction_data
}

for name, df in datasets.items():
    print(f"\n{name}:")
    print(f"Columns: {list(df.columns)}")



DataFrames loaded successfully!
campaign_desc shape: (30, 4)
campaign_table shape: (7208, 3)
coupon_redempt shape: (2318, 4)
coupon shape: (124548, 3)
product shape: (92353, 7)
transaction_data shape: (2595732, 12)
Headers for each dataset:

campaign_desc:
Columns: ['DESCRIPTION', 'CAMPAIGN', 'START_DAY', 'END_DAY']

campaign_table:
Columns: ['DESCRIPTION', 'household_key', 'CAMPAIGN']

coupon_redempt:
Columns: ['household_key', 'DAY', 'COUPON_UPC', 'CAMPAIGN']

coupon:
Columns: ['COUPON_UPC', 'PRODUCT_ID', 'CAMPAIGN']

product:
Columns: ['PRODUCT_ID', 'MANUFACTURER', 'DEPARTMENT', 'BRAND', 'COMMODITY_DESC', 'SUB_COMMODITY_DESC', 'CURR_SIZE_OF_PRODUCT']

transaction_data:
Columns: ['household_key', 'BASKET_ID', 'DAY', 'PRODUCT_ID', 'QUANTITY', 'SALES_VALUE', 'STORE_ID', 'RETAIL_DISC', 'TRANS_TIME', 'WEEK_NO', 'COUPON_DISC', 'COUPON_MATCH_DISC']


In [7]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92353 entries, 0 to 92352
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   PRODUCT_ID            92353 non-null  int64 
 1   MANUFACTURER          92353 non-null  int64 
 2   DEPARTMENT            92353 non-null  object
 3   BRAND                 92353 non-null  object
 4   COMMODITY_DESC        92353 non-null  object
 5   SUB_COMMODITY_DESC    92353 non-null  object
 6   CURR_SIZE_OF_PRODUCT  92353 non-null  object
dtypes: int64(2), object(5)
memory usage: 4.9+ MB


In [8]:
transaction_data['household_key'] = transaction_data['household_key'].astype(str)
transaction_data['STORE_ID'] = transaction_data['STORE_ID'].astype(str)
transaction_data['PRODUCT_ID'] = transaction_data['PRODUCT_ID'].astype(str)
transaction_data['BASKET_ID'] = transaction_data['BASKET_ID'].astype(str)

# FIXED: Use .astype(str) instead of str() for the entire column
coupon['COUPON_UPC'] = coupon['COUPON_UPC'].astype(str)  # Changed this line
coupon['PRODUCT_ID'] = coupon['PRODUCT_ID'].astype(str)
coupon['CAMPAIGN'] = coupon['CAMPAIGN'].astype(str)

campaign_desc['CAMPAIGN'] = campaign_desc['CAMPAIGN'].astype(str)

coupon_redempt['household_key'] = coupon_redempt['household_key'].astype(str)
coupon_redempt['COUPON_UPC'] = coupon_redempt['COUPON_UPC'].astype(str)
coupon_redempt['CAMPAIGN'] = coupon_redempt['CAMPAIGN'].astype(str)

campaign_table['CAMPAIGN'] = campaign_table['CAMPAIGN'].astype(str)
campaign_table['household_key'] = campaign_table['household_key'].astype(str)

product['PRODUCT_ID'] = product['PRODUCT_ID'].astype(str)
product['DEPARTMENT'] = product['DEPARTMENT'].astype(str)
product['MANUFACTURER'] = product['MANUFACTURER'].astype(str)

##3.Data Preparation

###3-1.Transaction

In [9]:
coupon_redempt= coupon_redempt.merge(
    coupon[['COUPON_UPC', 'PRODUCT_ID']],
    on='COUPON_UPC',
    how='left'
)

coupon= coupon.merge(
    campaign_desc[['START_DAY', 'END_DAY', 'CAMPAIGN']],
    on='CAMPAIGN',
    how='left'
)

coupon = coupon.merge(
    product[['PRODUCT_ID', 'DEPARTMENT', 'MANUFACTURER']],
    on='PRODUCT_ID',
    how='left'
)


In [10]:
# Create timestamp feature from DAY and TRANS_TIME
def create_timestamp(day, trans_time):
    base_date = datetime(2010, 3, 24)  # Arbitrary base date
    date_part = base_date + timedelta(days=int(day) - 1)

    # Convert TRANS_TIME to time
    hours = int(trans_time) // 100
    minutes = int(trans_time) % 100
    time_part = timedelta(hours=hours, minutes=minutes)

    return date_part + time_part

# Add timestamp to transaction_data
transaction_data['timestamp'] = transaction_data.apply(
    lambda row: create_timestamp(row['DAY'], row['TRANS_TIME']), axis=1
)

ts = transaction_data["timestamp"]
transaction_data["hour"]        = ts.dt.hour.astype("Int64")

transaction_data["dayofweek"]   = ts.dt.dayofweek.astype("Int64")    # 0=Mon .. 6=Sun
transaction_data["is_weekend"]  = transaction_data["dayofweek"].isin([5,6]).astype("int64")

transaction_data["day"]         = ts.dt.day.astype("Int64")
transaction_data["week"]        = ts.dt.isocalendar().week.astype("Int64")
transaction_data["month"]       = ts.dt.month.astype("Int64")
transaction_data["quarter"]     = ts.dt.quarter.astype("Int64")
transaction_data["year"]        = ts.dt.year.astype("Int64")

def make_part_of_day(hhmm_int):
    hh = hhmm_int // 100
    return (
        "night" if hh < 6 else
        "morning" if hh < 12 else
        "afternoon" if hh < 18 else
        "evening"
    )

transaction_data["part_of_day"] = transaction_data["TRANS_TIME"].astype(int).apply(make_part_of_day)
transaction_data = transaction_data.merge(
        product[['PRODUCT_ID', 'DEPARTMENT',"MANUFACTURER"]],
        on='PRODUCT_ID',
        how='left'
    )

transaction_data = transaction_data.merge(
    coupon_redempt,
    on=['household_key', 'DAY', 'PRODUCT_ID'],
    how='left'
)


In [11]:
transaction_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2598070 entries, 0 to 2598069
Data columns (total 26 columns):
 #   Column             Dtype         
---  ------             -----         
 0   household_key      object        
 1   BASKET_ID          object        
 2   DAY                int64         
 3   PRODUCT_ID         object        
 4   QUANTITY           int64         
 5   SALES_VALUE        float64       
 6   STORE_ID           object        
 7   RETAIL_DISC        float64       
 8   TRANS_TIME         int64         
 9   WEEK_NO            int64         
 10  COUPON_DISC        float64       
 11  COUPON_MATCH_DISC  float64       
 12  timestamp          datetime64[ns]
 13  hour               Int64         
 14  dayofweek          Int64         
 15  is_weekend         int64         
 16  day                Int64         
 17  week               Int64         
 18  month              Int64         
 19  quarter            Int64         
 20  year               Int64

In [12]:
RANDOM_STATE = 42
TOPK_CANDS = 50         # จำนวน candidates ต่อ query ที่จะป้อนเข้า ranker
EVAL_AT = [5, 10]          # NDCG@5, NDCG@10

In [13]:
transaction_data = transaction_data.sort_values(["household_key", "timestamp", "BASKET_ID"])
orders = (
    transaction_data.groupby(["household_key", "BASKET_ID"])["PRODUCT_ID"]
    .apply(lambda s: set(s.tolist()))
    .rename("items")
    .reset_index()
)

orders["next_items"] = orders.groupby("household_key")["items"].shift(-1)
orders = orders.dropna(subset=["next_items"]).reset_index(drop=True)

orders.head(3)

,household_key,BASKET_ID,items,next_items
0,1,27601281299,"{840361, 958046, 945805, 977545, 8069064, 1082...","{997025, 856942, 6773032, 7167962, 1049998, 92..."
1,1,27774192959,"{997025, 856942, 6773032, 7167962, 1049998, 92...","{991024, 1033142, 995242, 841266, 9527066, 865..."
2,1,28024266849,"{991024, 1033142, 995242, 841266, 9527066, 865...","{933303, 995242, 6514011, 945805, 857540, 9339..."


In [14]:
# === Cell 4: candidate generation (simple baseline) ===
# popularity ล่าสุดทั้งคอร์ปัส
pop_recent = (
    transaction_data["PRODUCT_ID"].value_counts()
    .head(1000)                # จำกัด universe เบื้องต้น
    .index.tolist()
)

def build_user_candidates(user_hist_items, topk=TOPK_CANDS):
    base = pop_recent.copy()

    # boost รายการที่ user เคยซื้อ (history)
    if user_hist_items:
        hist = list(user_hist_items)
        base = hist + [p for p in base if p not in user_hist_items]

    return base[:topk]



In [15]:
# === Cell 5 (FAST): build (query, candidate) rows with labels — per-household cumulative history ===

# 1) จัดเรียงลำดับภายใน household (แนะนำใช้เวลาจริง ถ้ามีคอลัมน์ timestamp ให้ใช้แทน BASKET_ID)
orders_fast = orders.sort_values(["household_key", "BASKET_ID"]).reset_index(drop=True)

# 2) สร้างคอลัมน์ 'hist_inclusive' = union ของ items ตั้งแต่อดีตถึงออเดอร์ปัจจุบัน (ต่อ household)
def _add_hist_inclusive(grp):
    hist = set()
    res  = []
    for s in grp["items"]:
        hist = hist | s  # รวมของเดิม + ของในตะกร้าปัจจุบัน (ตามตรรกะเดิมที่ใช้ <= t)
        res.append(hist.copy())
    out = grp.copy()
    out["hist_inclusive"] = res
    return out

orders_fast = (
    orders_fast
    .groupby("household_key", group_keys=False)
    .apply(_add_hist_inclusive)
    .reset_index(drop=True)
)

# 3) สร้างคู่ (query, candidate) แบบเชิงเส้น (ไม่มีการกรอง DataFrame ภายในลูป)
rows = []
append_row = rows.append  # ผูกฟังก์ชัน เพื่อลด overhead เล็กน้อย

for _, r in orders_fast.iterrows():
    user = r["household_key"]; t = r["BASKET_ID"]

    user_hist = r["hist_inclusive"]
    cands = build_user_candidates(user_hist, topk=TOPK_CANDS)

    positives = set(r["next_items"])

    # บังคับให้ positives ติดอยู่ใน candidates เสมอ แล้วค่อยตัดให้ไม่เกิน TOPK_CANDS
    if positives:
        cands = list(dict.fromkeys(list(positives) + [p for p in cands if p not in positives]))[:TOPK_CANDS]

    # สร้างแถวผลลัพธ์
    for pid in cands:
        append_row({
            "household_key": user,
            "BASKET_ID": int(t),
            "PRODUCT_ID": pid,
            "label": int(pid in positives)
        })

pair_df = pd.DataFrame(rows)
pair_df.head(3)

,household_key,BASKET_ID,PRODUCT_ID,label
0,1,27601281299,997025,1
1,1,27601281299,856942,1
2,1,27601281299,6773032,1


In [16]:
# === Cell 6: feature engineering (baseline) ===
# join ข้อมูลสินค้า
feat = pair_df.merge(product[["PRODUCT_ID","DEPARTMENT","BRAND"]], on="PRODUCT_ID", how="left")

# popularity ต่อสินค้า
pop_map = transaction_data["PRODUCT_ID"].value_counts().to_dict()
feat["feat_pop"] = feat["PRODUCT_ID"].map(pop_map).fillna(0).astype(float)

# ความถี่ที่ user เคยซื้อสินค้านั้น
user_item_ct = transaction_data.groupby(["household_key","PRODUCT_ID"]).size()
feat["feat_user_item_ct"] = feat.set_index(["household_key","PRODUCT_ID"]).index.map(user_item_ct).fillna(0).values

# one-hot เบื้องต้น (ปรับไปใช้ target encoding ก็ได้)
feat["DEPARTMENT"] = feat["DEPARTMENT"].fillna("UNK")
feat["BRAND"] = feat["BRAND"].fillna("UNK")
X_cats = pd.get_dummies(feat[["DEPARTMENT","BRAND"]], drop_first=True)

# รวมฟีเจอร์ตัวเลข + แคต
X_num = feat[["feat_pop","feat_user_item_ct"]].reset_index(drop=True)
X = pd.concat([X_num, X_cats.reset_index(drop=True)], axis=1)
y = feat["label"].values

# qid สำหรับจัดกลุ่ม (จำเป็นต่อ LGBMRanker)
feat["qid"] = feat["household_key"].astype(str) + "_" + feat["BASKET_ID"].astype(str)


In [17]:
# === Cell 7: group-aware split ===
qids = feat["qid"].unique()
q_train, q_valid = train_test_split(qids, test_size=0.2, random_state=RANDOM_STATE)

train_mask = feat["qid"].isin(q_train)
valid_mask = feat["qid"].isin(q_valid)

X_tr, y_tr = X[train_mask], y[train_mask]
X_va, y_va = X[valid_mask], y[valid_mask]

grp_tr = feat.loc[train_mask].groupby("qid").size().tolist()
grp_va = feat.loc[valid_mask].groupby("qid").size().tolist()


array(['1802_28282523073', '345_29019201656', '212_35469987090', ...,
       '2087_27534932822', '2221_33957201680', '2012_33994397762'],
      dtype=object)

In [21]:
# === Cell 8: train LGBMRanker (fixed: use callbacks instead of verbose) ===
import lightgbm as lgb

ranker = lgb.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="gbdt",
    num_leaves=63,
    learning_rate=0.05,
    n_estimators=500,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=RANDOM_STATE,
    # ถ้าต้องการลด warning จาก lightgbm เอง อาจใช้ verbosity=-1 (คนละเรื่องกับ progress log)
    # verbosity=-1,
)

# ใช้ callbacks แทน verbose
callbacks = [
    lgb.log_evaluation(period=50),                    # print metric ทุก ๆ 50 รอบ
    lgb.early_stopping(stopping_rounds=50,           # early stop ถ้าไม่ดีขึ้น 50 รอบ
                       first_metric_only=True)        # โฟกัสเมตริกตัวแรก (ndcg)
]

ranker.fit(
    X_tr, y_tr,
    group=grp_tr,
    eval_set=[(X_va, y_va)],
    eval_group=[grp_va],
    eval_at=EVAL_AT,          # เช่น [5,10] → NDCG@5, NDCG@10
    callbacks=callbacks       # << เปลี่ยนจาก verbose=50 มาเป็น callbacks
)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049855 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 449
[LightGBM] [Info] Number of data points in the train set: 10959350, number of used features: 34
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

LGBMRanker(colsample_bytree=0.8, learning_rate=0.05, metric='ndcg',
           n_estimators=500, num_leaves=63, objective='lambdarank',
           random_state=42, subsample=0.8)

In [22]:
# === Cell 9: inference & within-group ranking ===
valid_frame = feat[valid_mask].copy()
valid_frame["score"] = ranker.predict(X_va)

# จัดอันดับภายในกลุ่ม qid
valid_frame["rank"] = valid_frame.groupby("qid")["score"].rank(ascending=False, method="first")
topk = (valid_frame.sort_values(["qid","rank"])
                 .groupby("qid")
                 .head(10)[["qid","PRODUCT_ID","score","rank","label"]])

topk.head(10)


,qid,PRODUCT_ID,score,rank,label
6411,1000_27732872164,1082185,4.169619,1.0,1
6416,1000_27732872164,995242,2.258216,2.0,0
6421,1000_27732872164,1133018,1.963344,3.0,0
6446,1000_27732872164,1026118,1.945616,4.0,0
6433,1000_27732872164,823862,1.605656,5.0,0
6430,1000_27732872164,922127,1.593544,6.0,0
6404,1000_27732872164,901747,1.437108,7.0,1
6401,1000_27732872164,1006187,1.355284,8.0,1
6410,1000_27732872164,1139418,1.352534,9.0,1
6447,1000_27732872164,1131438,1.088137,10.0,0


In [26]:
# === Cell 10: simple Recall@K / Precision@K ===
def recall_precision_at_k(df, k=10):
    got = (df.sort_values(["qid","score"], ascending=[True, False])
             .groupby("qid").head(k))
    # recall@k = #positive ที่ดึงติด / #positive ทั้งหมดในกลุ่ม
    pos_per_q = df.groupby("qid")["label"].sum()
    hit_per_q = got.groupby("qid")["label"].sum()
    recall = (hit_per_q / pos_per_q.replace(0, np.nan)).mean()  # เฉลี่ยเฉพาะกลุ่มที่มี positive
    precision = got.groupby("qid")["label"].mean().mean()       # เฉลี่ย precision ต่อ qid
    return recall, precision

recall10, precision10 = recall_precision_at_k(valid_frame, k=10)
print(f"Recall@10={recall10:.4f}  Precision@10={precision10:.4f}")


Recall@10=0.5500  Precision@10=0.3619
